In [89]:
# Import data from Excel file
import pandas as pd

data = pd.read_excel('Entities Summary View_v1.xlsx', sheet_name='Entities Summary View', na_values=['NaN'])

In [1]:
# Import data from Airtable API
import configparser
import os
from pyairtable import Api

config = configparser.ConfigParser()
config.read("config.ini")
os.environ["AIRTABLE_API_KEY"] = config['krishna.udaiwal@cra-arc.gc.ca']['PAT']
base_id = config['airtable']['base_id']
table_name = config['airtable']['table_name']

api = Api(os.environ.get("AIRTABLE_API_KEY"))
table = api.table(base_id, table_name)
airtable_data = table.all()

In [10]:
table_cols = ['ID', 'Label', 'Connections - Entity From', 'Entity Short Form', 
              'Tags', 'Type', 'SubType', 'Description', 'OrgClass', 'OrgSubType', 
              'Portfolio', 'URL', 'Source of Entry', 'Admin Notes', 'Data Provider', 
              'Record Status', 'Last Modified By', 'Created', 
              'French Entity Full Name', 'French Short Form', 'French Description', 'French URL']

In [16]:
airtable_data[0]['fields']

{'Label': "Canada's Free Agents",
 'Type': ['rec1Ya3F8O9BtZZ1p'],
 'URL': 'https://wiki.gccollab.ca/Canada%27s_Free_Agents',
 'ID': 4730,
 'Tags': ['recTe4iB8nfwsZMmz'],
 'Connections - Entity To': ['recWSkrT1GkPvdYsg',
  'reccNyFnPwkOLPDTY',
  'recX6eUS7l6yuuvix',
  'recU62eJZLU8Hvg0T'],
 'French Entity Full Name': 'Agents libres du Canada',
 'French Description': "Ce programme, lancé en 2016, est un nouveau modèle de mobilisation de la main-d'œuvre. Il offre aux fonctionnaires la liberté de sélectionner un travail qui correspond à leurs compétences et à leurs intérêts, tout en leur permettant d’apporter une contribution qu'ils trouvent significative. Il permet aux gestionnaires de recourir rapidement et facilement aux meilleurs talents à l’affût des compétences émergentes et essentielles afin de répondre aux besoins à court terme de leur projet. Les Agents libres sont sélectionnés en fonction de leurs attributs utiles à la résolution de problèmes et des compétences qui sont en demand

In [ ]:
data_view = data[table_cols]
data_view = data_view.rename(columns={"Label": "defined_class_label",
                                        "ID":"defined_class",
                                        "Connections - Entity From": "parent_class",
                                        "Description": "definition",
                                        "URL": "definition_source"})
#df.rename(columns={"A": "a", "B": "c"})

In [90]:
# Clean up text data from columns

data['ID'] = "GCDE:"+data['ID'].astype(str).str.zfill(7)            # Add GCDE prefix and pad with zeros
desc = data['Description'].str.replace('[a-zA-Z0-9 .,]', '')[data['Description'].str.replace('[a-zA-Z0-9 .,]', '').notna()]
spec_chars = ""
for i in desc:
    if i == "":
        continue
#    print(i)
    for j in i:
        if j not in spec_chars:
            spec_chars += j

print(spec_chars)

#TODO Use above to create a list of special characters to remove, except for the ones we want to keep
# only allow alphanumeric characters, special characters: .,() and space
# '+-:()/\'–$&!;‑"|°?€=£·#~±%@[]'

# https://stackoverflow.com/questions/1276764/stripping-everything-but-alphanumeric-chars-from-a-string-in-python/1280823#1280823
delchars = ''.join(spec_chars for c in map(chr, range(256)) if not c.isalnum() and c not in ' .,+-:()/\'–$&!;‑"|°?€=£·#~±%@[]')
del_map = str.maketrans('', '', delchars)

def remove_special_characters(text):
    return text.translate(del_map)

# Remove new lines, leading and trailing whitespaces, and special characters
def fix_text(text):
    return text.str.replace('\n', ' ').apply(lambda x: remove_special_characters(x.strip()) if isinstance(x, str) else x) 

# Test
a = pd.Series(['a™', 'édf\nd', ' c   '])
print("Test:", a, a.str.replace('\n', ' ').apply(lambda x: remove_special_characters(x.strip()) if isinstance(x, str) else x), sep='\n')


+-:()_ /'–$’&!“”‘é;—•®ﬂÉ‑"|àè°	?â€™=£·#~±%@[]…\
Test:
0        a™
1    édf\nd
2      c   
dtype: object
0       a
1    df d
2       c
dtype: object


/tmp/ipykernel_1761218/734732074.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  desc = data['Description'].str.replace('[a-zA-Z0-9 .,]', '')[data['Description'].str.replace('[a-zA-Z0-9 .,]', '').notna()]


In [91]:
data['Description'] = fix_text(data['Description'])
data['French Description'] = fix_text(data['French Description'])
data['Admin Notes'] = fix_text(data['Admin Notes'])

In [92]:
def add_prefix(x):
    # takes lists: splits, pads with zeros, and adds GCDE prefix, then joins
    # takes strings: pads with zeros, and adds GCDE prefix
    if isinstance(x, list):
        return ",".join(["GCDE:"+i.astype(str).str.zfill(7) for i in x])
    return "GCDE:"+x.astype(str).str.zfill(7)
add_prefix(pd.Series([1,2,3]))

0    GCDE:0000001
1    GCDE:0000002
2    GCDE:0000003
dtype: object

In [93]:
def add_prefix(x):
    # takes lists: splits, pads with zeros, and adds GCDE prefix, then joins
    # takes strings: pads with zeros, and adds GCDE prefix
    if isinstance(x, (str, list)):
        #print("recogized as list")  6000
        y = ""
        #print(type(x), str(*x).split(","), "note3")
        for i in str(*x).split(", "):
            y = y+";"+("GCDE:"+i.zfill(7))
#            print(type(i), y, "Note2")
        return "{"+y[1:]+"}"
#        print(",".join(["GCDE:"+i.zfill(7) for i in x]))
#        return "{"+",".join(["GCDE:"+i.zfill(7) for i in x])+ "}"
    return "GCDE:"+str(x).zfill(7)

# Test multiple values
print(add_prefix(["1, 32, 3"]))
add_prefix(1)

{GCDE:0000001;GCDE:0000032;GCDE:0000003}


'GCDE:0000001'

In [94]:
# Update parent class to include GCDE prefix
# Split with GCDE prefixes

connections = []
for i in data['Connections - Entity From']:
    if isinstance(i, float):
        connections += [i]
    else:
        #print(type(i), i, "note1")
        connections += [add_prefix([i])]
    
data['Connections - Entity From'] = connections
#["GCDE:"+x.astype(str).str.zfill(7) for x in data['Connections - Entity From'].str.split(',') if isinstance(x, list)]

1623


In [96]:
data_view.to_csv('/home/agar2/Documents/1Projects/6Projects/GCDE_ontology/gcde_ontology/src/patterns/data/airtable_full.tsv', sep= "\t", index=False)

Label ->            defined_class_label
ID ->               defined_class
Connections - Entity From -> parent_class
Entity Short Form	
Tags	            
Type	            
SubType	
Description	->      definition
OrgClass	
OrgSubType	
Portfolio	
URL ->              definition_source
Source of Entry	
Admin Notes	
Data Provider	
Record Status	
Last Modified By	
Created	
French Entity Full Name	
French Short Form	
French Description	
French URL
